<a href="https://colab.research.google.com/github/Mathtzt/data-science/blob/main/apr_baseado_instancias/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objetivo e entendimento do problema

Neste notebook será analisado o dataset do kaggle [Cardiovascular Disease dataset](https://www.kaggle.com/datasets/sulianova/cardiovascular-disease-dataset) a respeito de um problema de classificação, a fim de descobrir possíveis pessoas com tendências a sofrer de problemas cardíacos. Para tal, essa análise utilizará processos e técnicas próprias para uma abordagem de aprendizado de máquina baseada em instâncias.

**Descrição dos dados**

Existem 3 tipos de variáveis:

* Objective: informação factual;
* Examination: resultados de exames médicos;
* Subjective: informação dada pelo paciente.

**Features:**

1. Age | Objective Feature | age | int (days)
2. Height | Objective Feature | height | int (cm)
3. Weight | Objective Feature | weight | float (kg)
4. Gender | Objective Feature | gender | categorical code | (1 - women, 2 - men)
5. Systolic blood pressure | Examination Feature | ap_hi | int
6. Diastolic blood pressure | Examination Feature | ap_lo | int
7. Cholesterol | Examination Feature | cholesterol | 1: normal, 2: above normal, 3: well above normal
8. Glucose | Examination Feature | gluc | 1: normal, 2: above normal, 3: well above normal
9. Smoking | Subjective Feature | smoke | binary
10. Alcohol intake | Subjective Feature | alco | binary
11. Physical activity | Subjective Feature | active | binary
12. Presence or absence of cardiovascular disease | Target Variable | cardio | binary

## Bibliotecas

In [1]:
import pandas as pd
import numpy as np

import plotly.offline as py
import plotly.graph_objs as go

from google.colab import files

## Importando Dados

**Uploading kaggle.json**

In [2]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


**Acessando dados do Kaggle**

In [3]:
%%capture
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

**Downloading**

In [4]:
%%capture
!kaggle datasets download sulianova/cardiovascular-disease-dataset

**Unzip**

In [5]:
%%capture
!unzip cardiovascular-disease-dataset.zip

## Overview dos dados

**Lendo os dados**

In [6]:
dataframe = pd.read_csv('cardio_train.csv', sep = ';')

In [7]:
dataframe.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


**Informações básicas**

In [8]:
print(f'Quantidade de amostras = {dataframe.shape[0]}')
print(f'Quantidade de colunas|variáveis = {dataframe.shape[1]}')

Quantidade de amostras = 70000
Quantidade de colunas|variáveis = 13


In [9]:
print(f'Tipo das variáveis: \n')
dataframe.dtypes

Tipo das variáveis: 



id               int64
age              int64
gender           int64
height           int64
weight         float64
ap_hi            int64
ap_lo            int64
cholesterol      int64
gluc             int64
smoke            int64
alco             int64
active           int64
cardio           int64
dtype: object

**Medidas descritivas**

In [10]:
dataframe.describe().round(2)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.0,70000.0
mean,49972.42,19468.87,1.35,164.36,74.21,128.82,96.63,1.37,1.23,0.09,0.05,0.8,0.5
std,28851.30,2467.25,0.48,8.21,14.40,154.01,188.47,0.68,0.57,0.28,0.23,0.4,0.5
min,0.00,10798.00,1.00,55.00,10.00,-150.00,-70.00,1.00,1.00,0.00,0.00,0.0,0.0
25%,25006.75,17664.00,1.00,159.00,65.00,120.00,80.00,1.00,1.00,0.00,0.00,1.0,0.0
50%,50001.50,19703.00,1.00,165.00,72.00,120.00,80.00,1.00,1.00,0.00,0.00,1.0,0.0
75%,74889.25,21327.00,2.00,170.00,82.00,140.00,90.00,2.00,1.00,0.00,0.00,1.0,1.0
max,99999.00,23713.00,2.00,250.00,200.00,16020.00,11000.00,3.00,3.00,1.00,1.00,1.0,1.0


**Análise de valores faltantes**

In [11]:
dataframe.isnull().sum()

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

## Análise Exploratória

### Métodos auxiliares

In [17]:
def barplot(xvalue, yvalue, title, xaxis, yaxis, fig_width = 1200):
  fig = go.Figure(data=[
      go.Bar(x = xvalue, y = yvalue, marker_color='lightsalmon')
  ])

  fig.update_layout(
      title=title,
      title_x=0.5,
      titlefont_size = 18,
      xaxis_tickfont_size=14,
      xaxis=dict(
        title=xaxis,
        titlefont_size=16,
        tickfont_size=14
      ),
      yaxis=dict(
          title=yaxis,
          titlefont_size=16,
          showticklabels=False,
          showgrid = False
      ),
      legend=dict(
          x=0,
          y=0,
          bgcolor='rgba(255, 255, 255, 0)',
          bordercolor='rgba(255, 255, 255, 0)'
      ),
      template='simple_white',
      width=fig_width
  )
  fig.update_traces(text=yvalue, texttemplate='%{text:.3s}', textposition='outside')
  fig.show()

In [40]:
def histogram(xvalue, title, xaxis, yaxis, fig_width = 1200):
  fig = go.Figure(data=[
        go.Histogram(x = xvalue, marker_color='lightsalmon')
    ])

  fig.update_layout(
      title=title,
      title_x=0.5,
      titlefont_size = 18,
      xaxis_tickfont_size=14,
      xaxis=dict(
        title=xaxis,
        titlefont_size=16,
        tickfont_size=14
      ),
      yaxis=dict(
          title=yaxis,
          titlefont_size=16,
          showticklabels=False,
          showgrid = False
      ),
      legend=dict(
          x=0,
          y=0,
          bgcolor='rgba(255, 255, 255, 0)',
          bordercolor='rgba(255, 255, 255, 0)'
      ),
      template='simple_white',
      width=fig_width
  )
  fig.show()

### Univariada

**Idade**

Como a idade é fornecida em dias, foi necessário converter para anos, a fim de facilitar a análise e entendimento. Tendo em vista que normalmente só contabilizamos o ano depois da data de nascimento, foi realizado o truncamento dos valores, removendo as casas decimais.

Além disso, anos bisextos não serão considerados.

In [12]:
dataframe['age_years'] = dataframe['age'] / 365
## convertendo para inteiro|truncamento
dataframe['age_years'] = dataframe['age_years'].astype(int)

In [19]:
temp_ = dataframe.value_counts(subset = 'age_years').reset_index(name = 'qtd')

barplot(xvalue = temp_['age_years'],
        yvalue = temp_['qtd'],
        title = 'Pessoas por idade',
        xaxis = 'Idade',
        yaxis = 'Quantidade')

Como observado, existem 4 amostras abaixo de 39 anos que estão destoando da distribuição. Apesar de não serem valores muito discrepantes, são registros raros no conjunto de dados e, por isso, serão removidos e analisados posteriormente.

**Sexo**

In [24]:
temp_ = dataframe.value_counts(subset = 'gender').reset_index(name = 'qtd')
temp_.replace({1: 'F', 2: 'M'}, inplace = True)

barplot(xvalue = temp_['gender'],
        yvalue = temp_['qtd'],
        title = 'Pessoas por sexo',
        xaxis = 'Sexo',
        yaxis = 'Quantidade',
        fig_width = 400)

O conjunto de dados apresenta um número mais elevado de pessoas do sexo feminino.

**Altura**

In [50]:
histogram(dataframe['height'] / 100,
          title = 'Distribuição das pessoas por altura',
          xaxis = 'Altura',
          yaxis = 'Frequência')

Possível notar a presença de valores muito abaixo da média da distribuição, assim como valores acima.

In [67]:
print(f"Quantidade de amostras com alturas abaixo de 1.20m: {dataframe[dataframe['height'] < 120].shape[0]}")
print(f"Quantidade de amostras com alturas acima de 2.0m: {dataframe[dataframe['height'] > 200].shape[0]}")

Quantidade de amostras com alturas abaixo de 1.20m: 52
Quantidade de amostras com alturas acima de 2.0m: 2


Dado que o conjunto de dados apresenta pessoas adultas, a existência de amostras com alturas abaixo de 1.20 e superiores a 2.0 será tratado como outlier, e removidos dos dados para treinamento.

**Peso**

In [69]:
histogram(dataframe['weight'],
          title = 'Distribuição das pessoas por peso',
          xaxis = 'Peso',
          yaxis = 'Frequência')

Possível notar a presença de pessoas com peso abaixo de 40kg e com mais de 140kg.

In [70]:
print(f"Quantidade de amostras com peso abaixo de 40kg: {dataframe[dataframe['weight'] < 40].shape[0]}")
print(f"Quantidade de amostras com peso acima de 140kg: {dataframe[dataframe['weight'] > 140].shape[0]}")

Quantidade de amostras com peso abaixo de 40kg: 52
Quantidade de amostras com peso acima de 140kg: 98


A manutenção de tais valores é aceitável, porém a fim de estabelecer limiares no conjunto de dados devido a baixa frequencia de ocorrências na faixas supracitadas, elas serão removidas para treinamento.

**Pressão sanguínea sistólica**

A pressão arterial sistólica (PAS), também conhecida como “pressão máxima”, se refere à pressão do sangue no momento que o coração se contrai para impulsionar o sangue para as artérias. Quanto mais o coração se contrai, maior é a pressão sistólica.

Ref: https://conteudo.omronbrasil.com/o-que-e-pressao-arterial-sistolica-2/

Como não é possível que a pressão arterial diastólica sejá maior que a sistólica, essas amostras serão removidas.

In [80]:
print(f"Serão removidas {dataframe[dataframe['ap_lo'] > dataframe['ap_hi']].shape[0]} amostras.")
dataframe = dataframe[dataframe['ap_lo'] < dataframe['ap_hi']]

Serão removidas 1234 amostras.


Dado que a PAS varia a partir de 90mmHg, e que valores acima de 180mmHg já são sinais de problemas graves, visto que o valor máximo normal é 140mmHg. Apenas amostras que estão entre [90-200] serão mantidas.

In [86]:
print(f"Serão removidas {dataframe[(dataframe['ap_hi'] < 90) | (dataframe['ap_hi'] > 200)].shape[0]} amostras.")
dataframe = dataframe[(dataframe['ap_hi'] >= 90) & (dataframe['ap_hi'] <= 200)]

Serão removidas 172 amostras.


In [87]:
histogram(dataframe['ap_hi'],
          title = 'Distribuição de pressão arterial sistólica',
          xaxis = 'PAS',
          yaxis = 'Frequência')

**Pressão sanguínea diastólica**

A pressão arterial diastólica (PAD) ou “pressão mínima” ocorre no início do ciclo cardíaco e se refere à capacidade de adaptação ao volume de sangue que o coração ejetou.

Dado que a PAD varia normalmente a partir de 60mmHg, e que valores acima de 110mmHg já são sinais de problemas graves. Apenas amostras que estão entre [60-120] serão mantidas.

In [90]:
print(f"Serão removidas {dataframe[(dataframe['ap_lo'] < 60) | (dataframe['ap_lo'] > 120)].shape[0]} amostras.")
dataframe = dataframe[(dataframe['ap_lo'] >= 60) & (dataframe['ap_lo'] <= 120)]

Serão removidas 179 amostras.


In [91]:
histogram(dataframe['ap_lo'],
          title = 'Distribuição de pressão arterial diastólica',
          xaxis = 'PAD',
          yaxis = 'Frequência')

**Colesterol**

In [97]:
temp_ = dataframe.value_counts(subset = 'cholesterol').reset_index(name = 'qtd')
temp_.replace({1: 'Normal', 2: 'Acima do Normal', 3: 'Muito acima do normal'}, inplace = True)

barplot(xvalue = temp_['cholesterol'],
        yvalue = temp_['qtd'],
        title = 'Níveis de colesterol',
        xaxis = '',
        yaxis = 'Qtd',
        fig_width = 800)

**Glicose**

In [98]:
temp_ = dataframe.value_counts(subset = 'gluc').reset_index(name = 'qtd')
temp_.replace({1: 'Normal', 2: 'Acima do Normal', 3: 'Muito acima do normal'}, inplace = True)

barplot(xvalue = temp_['gluc'],
        yvalue = temp_['qtd'],
        title = 'Níveis de glicose',
        xaxis = '',
        yaxis = 'Qtd',
        fig_width = 800)

**Tabagismo**

In [101]:
temp_ = dataframe.value_counts(subset = 'smoke').reset_index(name = 'qtd')
temp_.replace({0: 'Não-Fumante', 1: 'Fumante'}, inplace = True)

barplot(xvalue = temp_['smoke'],
        yvalue = temp_['qtd'],
        title = 'Tabagismo',
        xaxis = '',
        yaxis = 'Qtd',
        fig_width = 400)

**Alcoolismo**

In [102]:
temp_ = dataframe.value_counts(subset = 'alco').reset_index(name = 'qtd')
temp_.replace({0: 'Não', 1: 'Sim'}, inplace = True)

barplot(xvalue = temp_['alco'],
        yvalue = temp_['qtd'],
        title = 'Pessoas que bebem bebidas alcoolicas',
        xaxis = '',
        yaxis = 'Qtd',
        fig_width = 400)

**Prática de exercícios físicos**

In [103]:
temp_ = dataframe.value_counts(subset = 'active').reset_index(name = 'qtd')
temp_.replace({0: 'Não', 1: 'Sim'}, inplace = True)

barplot(xvalue = temp_['active'],
        yvalue = temp_['qtd'],
        title = 'Pessoas que praticam exercícios físicos',
        xaxis = '',
        yaxis = 'Qtd',
        fig_width = 400)

**Existência de problemas cardíacos (Ground Truth)**

In [104]:
temp_ = dataframe.value_counts(subset = 'cardio').reset_index(name = 'qtd')
temp_.replace({0: 'Não', 1: 'Sim'}, inplace = True)

barplot(xvalue = temp_['cardio'],
        yvalue = temp_['qtd'],
        title = 'Pessoas com problemas cardíacos',
        xaxis = '',
        yaxis = 'Qtd',
        fig_width = 400)